In [6]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

# Python的富比较方法

Python的富比较方法包括__lt__、__gt__分别表示：小于、大于，对应的操作运算符为:“<”、“>”。那么是否象普通数字运算一样，这两个方法之间存在互反的关系呢？
Python没有对__lt__、__gt__这两个方法之间有什么强制关系，但建议__lt__和__gt__为互反，开发者完全可以根据业务逻辑的需要自己定义。

我们来进行分析验证：
当自定义类中两个方法都定义了时，“<”、“>”分别调用__lt__和__gt__方法；


In [3]:
class Car:
    def __init__(self, carname, oilcp100km, price):
        self.carname = carname
        self.oilcp100km = oilcp100km
        self.price = price

    def __lt__(self, other):
        print("execute __lt__")
        return self.price < other.price


car1, car2, car3, car4 = Car('爱丽舍', 8, 10), Car('凯美瑞', 7, 27), Car('爱丽舍', 8, 10), Car('途观', 12, 27)
car1 > car2

execute __lt__


False

当自定义类中定义了__lt__方法，未定义__gt__方法时，进行”>”比较也是调用__lt__方法，只是对调用值求反。
案例：

In [5]:
class Car2:
    def __init__(self, carname, oilcp100km, price):
        self.carname = carname
        self.oilcp100km = oilcp100km
        self.price = price

    def __gt__(self, other):
        print("execute __gt__")
        return self.price > other.price


car1, car2, car3, car4 = Car2('爱丽舍', 8, 10), Car2('凯美瑞', 7, 27), Car2('爱丽舍', 8, 10), Car2('途观', 12, 27)
car1 < car2
car1 > car2

execute __gt__


True

execute __gt__


False

通过以上案例，我们知道当__lt__、__gt__方法都定义了时，“<”、“>”分别调用__lt__和__gt__方法，
当一个定义另一个未定义时，未定义的操作执行时会调用已经定义的方法求反。这个与__eq__和__ne__的关系还是有较大的不同。

In [12]:
class Customer:
    def __init__(self, num, x):
        # 表示到来的客户编号为 num ,优先级为 x
        self.num = num
        self.x = x

    def __lt__(self, other):
        # 如果同时有多位优先级相同且最高的客户，则按照先来后到的顺序办理
        if self.x == other.x:
            return self.num < other.num
        else:
            return self.x < other.x


cus1 = Customer(1, 6)
cus2 = Customer(2, 6)
cus3 = Customer(2, 2)
cus1 < cus2
cus1 > cus3

True

True

In [1]:
# 并集查结构
class UnionFindSet:
    def __init__(self, n):  # 1。初始化0-n的父节点，和没有更改父节点的数量
        self.fa = [i for i in range(n)]  # 0-n各个索引对应的父节点，初始化是索引本身
        self.count = n  # 没有更新父节点的数量，初始化为数组本身长度

    def find(self, x):  # 2。找坐标的父节点:
        if x != self.fa[x]:
            # 2.1 如果x的父节点前面已经更改过，就找x的父父站点，并返回她的父父节点
            # 比如 (0, 1), (0, 3), (2, 3): 第二次查找3的父节点，应该返回0的父节点，也就是0
            self.fa[x] = self.find(self.fa[x])
            return self.fa[x]
        return x  # 2.2 如果x的父节点没更改过，正常返回索引本身

    def union(self, x, y):  # 3。更新坐标的父节点
        x_fa = self.find(x)
        y_fa = self.find(y)

        if x_fa != y_fa:
            self.fa[y_fa] = x_fa  # (0, 1):将1的父节点改为0
            self.count -= 1  # count记录没有更新过父节点的数量


ufs = UnionFindSet(3)
print(ufs)

# 类属性与方法
## 类的方法
在类的内部，使用 def 关键字来定义一个方法，与一般函数定义不同，类方法必须包含参数 self，且为第一个参数，self 代表的是类的实例。
self 的名字并不是规定死的，也可以使用 this，但是最好还是按照约定使用 self。

## 类的私有属性
__private_attrs：两个下划线开头，声明该属性为私有，不能在类的外部被使用或直接访问。在类内部的方法中使用时 self.__private_attrs。
## 类的私有方法
__private_method：两个下划线开头，声明该方法为私有方法，只能在类的内部调用 ，不能在类的外部调用。self.__private_methods。

In [8]:
class JustCounter:
    __secretCount = 0  # 私有变量
    publicCount = 0  # 公开变量

    def count(self):
        self.__secretCount += 1
        self.publicCount += 1
        # print (self.__secretCount)


counter = JustCounter()
# counter.count()
counter.count()
print(counter.publicCount)  # 1
print(counter.__secretCount)  # 报错，实例不能访问私有变量

1


AttributeError: 'JustCounter' object has no attribute '__secretCount'

# 全局变量
在Python中，内部类无法使用 global 关键字修改外部类变量。

global 关键字用于声明一个变量是全局变量，可以在函数或模块中的任何位置使用。
但是，内部类的作用域被限制在外部类中，因此无法使用 global 关键字来修改外部类变量。

如果要修改外部类变量，可以使用 self 关键字来引用外部类实例，并直接修改实例变量。例如：

In [2]:
class OuterClass:
    def __init__(self):
        self.count = 0

    class InnerClass:
        def __init__(self, outer):
            outer.count += 1


outer = OuterClass()
print(outer.count)  # 输出0
inner1 = outer.InnerClass(outer)
print(outer.count)  # 输出1
inner2 = outer.InnerClass(outer)
print(outer.count)  # 输出2

0
1
2


In [ ]:
class OuterClass:
    def __init__(self):
        self.count = 0

    class InnerClass:
        def __init__(self):
            # global self.count # 报错
            pass

# 在Python中，可以在类中使用类变量来实现全局变量的效果。

- 类变量是在类定义中定义的变量，它们是类的所有实例共享的。
- 可以通过类名或实例来访问类变量，并且可以在类的任何方法中修改类变量的值。
下面是一个示例代码

In [13]:
class MyClass:
    global_var = 0  # 定义类变量

    def __init__(self):
        self.instance_var = 0  # 定义实例变量

    def class_method(self):
        MyClass.global_var += 1  # 类方法中修改全局变量

    def instance_method(self):
        self.instance_var += 1  # 实例方法中修改实例变量
        MyClass.global_var += 1  # 实例方法中修改全局变量

    class InnerClass:
        def inner_method(self):
            MyClass.global_var += 1  # 内部类中修改全局变量


MyClass.global_var  # 访问类变量
obj = MyClass()
obj.global_var  # 访问类变量
obj.instance_var  # 访问实例变量
MyClass.global_var = 1  # 修改类变量
obj.global_var = 2  # 修改类变量
obj.instance_var = 3  # 修改实例变量

print(MyClass.global_var) # 1
print(obj.global_var) # 2

cls =  MyClass.InnerClass().inner_method()
print(MyClass.global_var) # 2

0

0

0

1
2
2
